# Exercise 2#

## Task 1 ##


a) Tokenize the sentences in fileyelp_polarity.txt, removing punctuation.

In [1]:
import string
import collections, functools, operator 
import pandas as pd
import numpy as np
import os

def read_file(filename):
    df = pd.read_csv(filename, sep="\t", header=None)
    df = df.assign(tokens =df[0].apply(lambda x : x.translate(str.maketrans('', '', string.punctuation)).lower().split()))
    return df

df = read_file("yelp_polarity.txt")
display(df)

,0,1,tokens
0,Wow... Loved this place.,1,"[wow, loved, this, place]"
1,Crust is not good.,0,"[crust, is, not, good]"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nasty]"
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an..."
...,...,...,...
995,I think food should have flavor and texture an...,0,"[i, think, food, should, have, flavor, and, te..."
996,Appetite instantly gone.,0,"[appetite, instantly, gone]"
997,Overall I was not impressed and would not go b...,0,"[overall, i, was, not, impressed, and, would, ..."
998,"The whole experience was underwhelming, and I ...",0,"[the, whole, experience, was, underwhelming, a..."


The file is read using pandas and saved in the dataframe df. A function is then applied to all of the sentences using the apply() funciton. The lambda-function used first removes punctuation, then applies lower() and split().

b) Calculate token and bigram counts

In [2]:
def tokens_to_token_freq(tokens):
    dictionary = dict.fromkeys(tokens)
    for i in list(dictionary):
        count = 0
        for j in tokens:
            if j==i:
                count += 1
        dictionary[i] = count
    return dictionary

def bigram_list(sent):
    bigrams=[]
    for i in range(0, len(sent) - 1):
        bigrams.append(sent[i] + ' ' + sent[i + 1])
    return bigrams

df = df.assign(bigrams = df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
df = df.assign(token_count = df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
df = df.assign(bigram_count = df["bigrams"].apply(lambda x : tokens_to_token_freq(x)))
display(df)

,0,1,tokens,bigrams,token_count,bigram_count
0,Wow... Loved this place.,1,"[wow, loved, this, place]","{'wow loved': None, 'loved this': None, 'this ...","{'wow': 1, 'loved': 1, 'this': 1, 'place': 1}","{'wow loved': 1, 'loved this': 1, 'this place'..."
1,Crust is not good.,0,"[crust, is, not, good]","{'crust is': None, 'is not': None, 'not good':...","{'crust': 1, 'is': 1, 'not': 1, 'good': 1}","{'crust is': 1, 'is not': 1, 'not good': 1}"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nasty]","{'not tasty': None, 'tasty and': None, 'and th...","{'not': 1, 'tasty': 1, 'and': 1, 'the': 1, 'te...","{'not tasty': 1, 'tasty and': 1, 'and the': 1,..."
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho...","{'stopped by': None, 'by during': None, 'durin...","{'stopped': 1, 'by': 1, 'during': 1, 'the': 1,...","{'stopped by': 1, 'by during': 1, 'during the'..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an...","{'the selection': None, 'selection on': None, ...","{'the': 3, 'selection': 1, 'on': 1, 'menu': 1,...","{'the selection': 1, 'selection on': 1, 'on th..."
...,...,...,...,...,...,...
995,I think food should have flavor and texture an...,0,"[i, think, food, should, have, flavor, and, te...","{'i think': None, 'think food': None, 'food sh...","{'i': 1, 'think': 1, 'food': 1, 'should': 1, '...","{'i think': 1, 'think food': 1, 'food should':..."
996,Appetite instantly gone.,0,"[appetite, instantly, gone]","{'appetite instantly': None, 'instantly gone':...","{'appetite': 1, 'instantly': 1, 'gone': 1}","{'appetite instantly': 1, 'instantly gone': 1}"
997,Overall I was not impressed and would not go b...,0,"[overall, i, was, not, impressed, and, would, ...","{'overall i': None, 'i was': None, 'was not': ...","{'overall': 1, 'i': 1, 'was': 1, 'not': 2, 'im...","{'overall i': 1, 'i was': 1, 'was not': 1, 'no..."
998,"The whole experience was underwhelming, and I ...",0,"[the, whole, experience, was, underwhelming, a...","{'the whole': None, 'whole experience': None, ...","{'the': 1, 'whole': 1, 'experience': 1, 'was':...","{'the whole': 1, 'whole experience': 1, 'exper..."


c) Print the 10 most frequent bigrams and their frequency

In [3]:
def frequency(column):
    frequency = dict(functools.reduce(operator.add, 
         map(collections.Counter, column)))
    frequency = sorted(frequency.items(), key=lambda x: x[1])
    return(list(frequency))

bigrams = frequency(df["bigram_count"])
print(bigrams[len(bigrams) - 10 : ])

[('on the', 23), ('the service', 24), ('service was', 26), ('and i', 27), ('of the', 29), ('food was', 32), ('and the', 38), ('it was', 41), ('the food', 47), ('this place', 72)]


d.) Print a bigram matrix from the 10 most frequent tokens

In [15]:
token_frequency = frequency(df["token_count"])

def bigram_matrix(token_list, bigram_frequencies):
    matrix = np.zeros((len(token_list), len(token_list)))
    for i in range(0, len(matrix)):
        for j in range(0, len(matrix)):
            key_str = token_list[i][0] + " " + token_list[j][0]
            bigram_frequencies = dict(bigram_frequencies)
            if key_str in bigram_frequencies:
                matrix[i, j] = bigram_frequencies[key_str]          
    return matrix
freq_matrix = bigram_matrix(token_frequency[len(token_frequency) - 10 : ], bigrams)
token_strings = dict(token_frequency[len(token_frequency) - 10 : ]).keys()

display(pd.DataFrame(freq_matrix, token_strings, token_strings))

,of,it,this,is,to,a,was,i,and,the
of,0.0,1.0,4.0,0.0,0.0,10.0,0.0,0.0,0.0,29.0
it,0.0,0.0,0.0,10.0,2.0,1.0,41.0,2.0,3.0,0.0
this,0.0,0.0,0.0,20.0,1.0,0.0,10.0,0.0,0.0,0.0
is,1.0,0.0,0.0,0.0,1.0,16.0,1.0,0.0,0.0,10.0
to,0.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,1.0,11.0
a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
was,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,13.0
i,0.0,0.0,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0
and,0.0,17.0,4.0,0.0,0.0,3.0,5.0,27.0,0.0,38.0
the,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Task 2

Using the code of Task 1:

a.) Extract top 20 most frequent bigrams of all positive and negative documents in folder polarity




In [16]:
pos_docs = os.listdir("./polarity/pos")
pos_df = read_file("./polarity/pos/" + pos_docs[0])
for doc in pos_docs[1:]:
    pos_df = pos_df.append(read_file("./polarity/pos/" + doc))

pos_df = pos_df.assign(bigrams = pos_df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
pos_df = pos_df.assign(token_count = pos_df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
pos_df = pos_df.assign(bigram_count = pos_df["bigrams"].apply(lambda x : tokens_to_token_freq(x)))

pos_bigrams = frequency(pos_df["bigram_count"])
print(pos_bigrams[len(pos_bigrams) - 20:])

neg_docs = os.listdir("./polarity/neg")
neg_df = read_file("./polarity/neg/" + neg_docs[0])
for doc in neg_docs[1:]:
    neg_df = neg_df.append(read_file("./polarity/neg/" + doc))

print()
neg_df = neg_df.assign(neg_bigrams = neg_df["tokens"].apply(lambda x : dict.fromkeys(bigram_list(x))))
neg_df = neg_df.assign(token_count = neg_df["tokens"].apply(lambda x : tokens_to_token_freq(x)))
neg_df = neg_df.assign(bigram_count = neg_df["neg_bigrams"].apply(lambda x : tokens_to_token_freq(x)))
# print(neg_df)
neg_bigrams = frequency(neg_df["bigram_count"])
print(neg_bigrams[len(neg_bigrams) - 20:])
# print(neg_df[0])

[('youve got', 4), ('for the', 4), ('jackie chan', 4), ('this film', 5), ('as the', 5), ('and the', 5), ('from the', 5), ('it is', 5), ('the movie', 5), ('the first', 5), ('is a', 6), ('to be', 7), ('is the', 7), ('with a', 7), ('the film', 8), ('with the', 8), ('by the', 8), ('the shark', 9), ('of the', 14), ('in the', 24)]

[('most of', 4), ('they are', 4), ('of a', 5), ('and the', 5), ('from the', 5), ('by the', 5), ('is not', 5), ('in a', 6), ('with the', 6), ('for the', 6), ('of course', 6), ('the movie', 6), ('it is', 6), ('film is', 6), ('on the', 7), ('to be', 8), ('that the', 8), ('in the', 8), ('the film', 9), ('of the', 10)]


b.) Calculate their probability as aforementioned

In [17]:
top_pos_bigrams = pos_bigrams[len(pos_bigrams) - 20:]
top_neg_bigrams = neg_bigrams[len(neg_bigrams) - 20:]

def calculate_probabilty(all_bigrams, bigrams):
    probability = []
    for x in bigrams:
        probability.append((x[0], x[1]/sum(dict(all_bigrams).values())))
    return probability

neg_probability = calculate_probabilty(neg_bigrams, top_neg_bigrams)
print("Negative Bigram Probabilities")
print(neg_probability)
print()

pos_probability = calculate_probabilty(pos_bigrams, top_pos_bigrams)
print("Positive Bigram Probabilities")
print(pos_probability)

Negative Bigram Probabilities
[('most of', 0.0016326530612244899), ('they are', 0.0016326530612244899), ('of a', 0.0020408163265306124), ('and the', 0.0020408163265306124), ('from the', 0.0020408163265306124), ('by the', 0.0020408163265306124), ('is not', 0.0020408163265306124), ('in a', 0.0024489795918367346), ('with the', 0.0024489795918367346), ('for the', 0.0024489795918367346), ('of course', 0.0024489795918367346), ('the movie', 0.0024489795918367346), ('it is', 0.0024489795918367346), ('film is', 0.0024489795918367346), ('on the', 0.002857142857142857), ('to be', 0.0032653061224489797), ('that the', 0.0032653061224489797), ('in the', 0.0032653061224489797), ('the film', 0.003673469387755102), ('of the', 0.004081632653061225)]

Positive Bigram Probabilities
[('youve got', 0.001234186979327368), ('for the', 0.001234186979327368), ('jackie chan', 0.001234186979327368), ('this film', 0.0015427337241592102), ('as the', 0.0015427337241592102), ('and the', 0.0015427337241592102), ('from

c.) Write a script to save in a file the most probably bigrams of both classes and their probabilites separated by tabs

In [64]:
neg_prob = dict(neg_probability)
pos_prob = dict(pos_probability)

df_neg = pd.DataFrame(neg_prob.values(),neg_prob.keys())
df_neg.columns = ['neg prob']
df_pos =  pd.DataFrame(pos_prob.values(), pos_prob.keys())
df_pos.columns = ['pos prob']
df_total = pd.concat([df_neg, df_pos], axis=1, sort=True)

display(df_total)
df.to_csv("test1.txt", sep="\t")

,neg prob,pos prob
and the,0.002041,0.001543
as the,NaN,0.001543
by the,0.002041,0.002468
film is,0.002449,NaN
for the,0.002449,0.001234
from the,0.002041,0.001543
in a,0.002449,NaN
in the,0.003265,0.007405
is a,NaN,0.001851
is not,0.002041,NaN


d.) Compare the output of both classes and write an analysis of your obersvations